# Pinecone DB (프리티어 사용)

In [15]:
#!pip install pinecone
#!pip install python-dotenv

In [16]:
from pinecone import Pinecone, ServerlessSpec
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv('API_KEY')
pc = Pinecone(api_key=api_key)
index_name = 'reviewtext'

pc.create_index(
    name=index_name,
    dimension=384,
    metric='cosine',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
        )
    )

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-01', 'X-Cloud-Trace-Context': '88dbd1f37dc761f4ff69e271968715af', 'Date': 'Mon, 10 Mar 2025 07:15:26 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [17]:
reviews = [
    {"id": "1", "text": "이 제품 정말 좋아요! 성능이 만족스러워요!", "sentiment": "positive"},
    {"id": "2", "text": "별로에요. 배터리가 너무 빨리 닳아요.", "sentiment": "negative"},
    {"id": "3", "text": "가격 대비 품질이 훌륭해요. 추천합니다!", "sentiment": "positive"},
    {"id": "4", "text": "배송이 빨라서 좋았어요. 포장도 튼튼하게 잘 되어 있었습니다.", "sentiment": "positive"},
    {"id": "5", "text": "사용하다가 한 달 만에 고장났어요. 너무 실망스럽습니다.", "sentiment": "negative"},
    {"id": "6", "text": "디자인이 세련되고 기능도 다양해서 만족합니다.", "sentiment": "positive"},
    {"id": "7", "text": "설명서가 너무 복잡하고 이해하기 어려워요.", "sentiment": "negative"},
    {"id": "8", "text": "화질이 선명하고 음질도 좋아요. 구매 잘한 것 같습니다!", "sentiment": "positive"},
    {"id": "9", "text": "고객 서비스가 너무 불친절해요. 문의해도 제대로 답변을 못 받았습니다.", "sentiment": "negative"},
    {"id": "10", "text": "생각보다 크기가 작아서 당황했어요. 제품 설명에 크기를 정확히 표시해주세요.", "sentiment": "negative"},
    {"id": "11", "text": "아이가 사용하기에 안전하고 편리해서 좋아요.", "sentiment": "positive"},
    {"id": "12", "text": "색상이 사진과 많이 달라요. 실망했습니다.", "sentiment": "negative"},
    {"id": "13", "text": "기능이 다양하고 사용하기 쉬워서 매우 만족합니다.", "sentiment": "positive"},
    {"id": "14", "text": "튼튼하고 내구성이 좋아 오래 사용할 수 있을 것 같아요.", "sentiment": "positive"},
    {"id": "15", "text": "소음이 너무 심해요. 밤에는 사용하기 어렵습니다.", "sentiment": "negative"},
    {"id": "16", "text": "인터넷 연결이 자주 끊겨서 불편해요.", "sentiment": "negative"},
    {"id": "17", "text": "가성비 최고! 이 가격에 이런 퀄리티라니 놀랍습니다.", "sentiment": "positive"},
    {"id": "18", "text": "배터리 지속 시간이 길어서 외출할 때 유용해요.", "sentiment": "positive"},
    {"id": "19", "text": "앱 호환성이 좋지 않아 사용하는 데 제한이 많아요.", "sentiment": "negative"},
    {"id": "20", "text": "화면이 자주 멈춰서 답답해요. 소프트웨어 업데이트가 필요합니다.", "sentiment": "negative"}
]

In [20]:
import pinecone as pc
from sentence_transformers import SentenceTransformer

pc = Pinecone(api_key=api_key)
idx = pc.Index(index_name)
model = SentenceTransformer('all-MiniLM-L6-v2')

In [21]:
for review in reviews:
    review_embed = model.encode(review['text']).tolist()
    idx.upsert([
        (review['id'], 
         review_embed,
            {
                'text': review['text'], 
                'sentiment': review['sentiment']
            }
        )
    ])

In [25]:
query_text = '이 노트북의 성능은 괜찮은가요?'
query_embed = model.encode(query_text).tolist()

results = idx.query(
    vector=query_embed,
    top_k=3,
    include_metadata=True    
)

results

{'matches': [{'id': '2',
              'metadata': {'sentiment': 'negative',
                           'text': '별로에요. 배터리가 너무 빨리 닳아요.'},
              'score': 0.711045802,
              'values': []},
             {'id': '12',
              'metadata': {'sentiment': 'negative',
                           'text': '색상이 사진과 많이 달라요. 실망했습니다.'},
              'score': 0.709889293,
              'values': []},
             {'id': '14',
              'metadata': {'sentiment': 'positive',
                           'text': '튼튼하고 내구성이 좋아 오래 사용할 수 있을 것 같아요.'},
              'score': 0.663241267,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}

In [28]:
for review in results['matches']:
    metadata = review['metadata']
    print(metadata['text'], f"(감성: {metadata['sentiment']})")

별로에요. 배터리가 너무 빨리 닳아요. (감성: negative)
색상이 사진과 많이 달라요. 실망했습니다. (감성: negative)
튼튼하고 내구성이 좋아 오래 사용할 수 있을 것 같아요. (감성: positive)


In [30]:
query_text = '배터리도 빨리 닳고 벽돌보다 무거워요. 비추천.'
query_embed = model.encode(query_text).tolist()

results = idx.query(
    vector=query_embed,
    top_k=5,
    include_metadata=True    
)

results

{'matches': [{'id': '7',
              'metadata': {'sentiment': 'negative',
                           'text': '설명서가 너무 복잡하고 이해하기 어려워요.'},
              'score': 0.878002405,
              'values': []},
             {'id': '1',
              'metadata': {'sentiment': 'positive',
                           'text': '이 제품 정말 좋아요! 성능이 만족스러워요!'},
              'score': 0.814241886,
              'values': []},
             {'id': '2',
              'metadata': {'sentiment': 'negative',
                           'text': '별로에요. 배터리가 너무 빨리 닳아요.'},
              'score': 0.798072636,
              'values': []},
             {'id': '3',
              'metadata': {'sentiment': 'positive',
                           'text': '가격 대비 품질이 훌륭해요. 추천합니다!'},
              'score': 0.780082047,
              'values': []},
             {'id': '9',
              'metadata': {'sentiment': 'negative',
                           'text': '고객 서비스가 너무 불친절해요. 문의해도 제대로 답변을 못 받았습니다.'},
              'score': 0.7

In [36]:
sentiment_counts = {
    'positive': 0,
    'negative': 0    
}

for review in results['matches']:
    sentiment = metadata = review['metadata']['sentiment']
    sentiment_counts[sentiment] += 1

print(f"리뷰 분석 결과: {'positive' if sentiment_counts['positive'] > sentiment_counts['negative'] else 'negative'}")

리뷰 분석 결과: negative
